In [4]:
import os
import json
import wave  # <--- 新增: 导入wave库来处理wav文件

def create_transcription_json(audio_folder_path, transcription_txt_path, output_json_path):
    """
    根据音频文件和转录文本文件，构建语音识别的转录JSON Lines文件。
    此版本新增了音频时长的计算。

    Args:
        audio_folder_path (str): 存放 .wav 音频文件的根文件夹路径。
        transcription_txt_path (str): 包含转录文本的 .txt 文件路径。
        output_json_path (str): 输出的 .jsonl 文件路径。
    """
    # --- 步骤 1: 读取转录文本文件并存入字典 ---
    print(f"正在读取转录文件: {transcription_txt_path}...")
    transcriptions = {}
    try:
        with open(transcription_txt_path, 'r', encoding='utf-8') as f:
            for line in f:
                line = line.strip()
                if not line:
                    continue
                
                parts = line.split(' ', 1)
                if len(parts) == 2:
                    key, sentence = parts
                    transcriptions[key] = sentence
                else:
                    print(f"警告：跳过格式不正确的行 -> {line}")

    except FileNotFoundError:
        print(f"错误：找不到转录文件 '{transcription_txt_path}'。请检查路径是否正确。")
        return
    
    print(f"成功读取 {len(transcriptions)} 条转录数据。")

    # --- 步骤 2 & 3 & 4 & 5: 遍历音频文件，匹配、处理并写入JSON ---
    print(f"正在遍历音频文件夹: {audio_folder_path}...")
    
    match_count = 0
    
    try:
        with open(output_json_path, 'w', encoding='utf-8') as out_f:
            for root, _, files in os.walk(audio_folder_path):
                for filename in files:
                    if filename.lower().endswith('.wav'):
                        base_name = os.path.splitext(filename)[0]
                        
                        if '-' in base_name:
                            key = base_name.split('-')[-1].strip()
                            
                            if key in transcriptions:
                                match_count += 1
                                
                                original_sentence = transcriptions[key]
                                modified_sentence = original_sentence.replace(' ', '་')
                                full_audio_path = os.path.abspath(os.path.join(root, filename))

                                # --- V--- 新增代码块：计算音频时长 ---V---
                                duration = 0.0
                                try:
                                    # 使用 with 语句确保文件能被正确关闭
                                    with wave.open(full_audio_path, 'rb') as wav_file:
                                        frames = wav_file.getnframes()
                                        rate = wav_file.getframerate()
                                        if rate > 0:
                                            duration = frames / float(rate)
                                except Exception as e:
                                    print(f"警告：无法读取音频文件 '{filename}' 的时长。错误: {e}")
                                
                                # 保留两位小数
                                duration = round(duration, 2)
                                # --- A--- 新增代码块结束 ---A---
                                
                                # <--- 修改: 在JSON结构中添加 "duration" 字段
                                data = {
                                    "audio": {"path": full_audio_path},
                                    "sentence": modified_sentence,
                                    "duration": duration 
                                }
                                
                                json_line = json.dumps(data, ensure_ascii=False)
                                out_f.write(json_line + '\n')
                            else:
                                print(f"警告：音频文件 '{filename}' 对应的键 '{key}' 在转录文件中未找到。")
    
    except Exception as e:
        print(f"处理过程中发生错误: {e}")
        return

    print("=" * 30)
    print("处理完成！")
    print(f"总共找到 {match_count} 个匹配的音频文件并已生成JSON。")
    print(f"结果已保存至: {output_json_path}")



In [5]:
# --- 如何使用 ---

if __name__ == "__main__":
    # 1. 请将这里的路径修改为您自己的实际路径
    # 音频文件所在的根文件夹
    audio_directory = "/media/ubuntu/3.2TB/code/icefall/egs/xbmu_amdo31_500+writtenText/ASR/download/xbmu_amdo31/data/wav/test" 
    # 转录文本文件
    transcription_file = "/media/ubuntu/3.2TB/code/icefall/egs/xbmu_amdo31_500+writtenText/ASR/download/xbmu_amdo31/data/transcript/transcript_clean.txt"
    # 输出的JSON Lines文件名
    output_file = "./output_transcription.jsonl"

    # 2. 运行主函数
    create_transcription_json(audio_directory, transcription_file, output_file)

正在读取转录文件: /media/ubuntu/3.2TB/code/icefall/egs/xbmu_amdo31_500+writtenText/ASR/download/xbmu_amdo31/data/transcript/transcript_clean.txt...
警告：跳过格式不正确的行 -> TAS_13_13298
警告：跳过格式不正确的行 -> TAS_143_19101
警告：跳过格式不正确的行 -> TAS_143_19118
警告：跳过格式不正确的行 -> TAS_143_19192
警告：跳过格式不正确的行 -> TAS_143_19211
警告：跳过格式不正确的行 -> TAS_155_23618
警告：跳过格式不正确的行 -> TAS_155_23968
警告：跳过格式不正确的行 -> TAS_158_25044
警告：跳过格式不正确的行 -> TAS_159_25244
警告：跳过格式不正确的行 -> TAS_159_25463
警告：跳过格式不正确的行 -> TAS_160_26267
警告：跳过格式不正确的行 -> TAS_164_27709
警告：跳过格式不正确的行 -> TAS_174_31725
警告：跳过格式不正确的行 -> TAS_175_32193
警告：跳过格式不正确的行 -> TAS_182_35384
警告：跳过格式不正确的行 -> TAS_230_54887
警告：跳过格式不正确的行 -> TAS_345_103440
警告：跳过格式不正确的行 -> TAS_4_108762
警告：跳过格式不正确的行 -> TAS_42_109906
警告：跳过格式不正确的行 -> TAS_58_116786
警告：跳过格式不正确的行 -> TAS_65_120092
警告：跳过格式不正确的行 -> TAS_85_128284
警告：跳过格式不正确的行 -> TAS_87_128869
警告：跳过格式不正确的行 -> TAS_93_131175
成功读取 838246 条转录数据。
正在遍历音频文件夹: /media/ubuntu/3.2TB/code/icefall/egs/xbmu_amdo31_500+writtenText/ASR/download/xbmu_amdo31/data/wav/test...
警告：音

In [6]:
import os

def split_jsonl_file(input_file_path, output_file_1_path, output_file_2_path):
    """
    将一个 JSON Lines (.jsonl) 文件平均分割成两个文件。

    Args:
        input_file_path (str): 需要被分割的原始 .jsonl 文件路径。
        output_file_1_path (str): 分割后第一个文件的保存路径。
        output_file_2_path (str): 分割后第二个文件的保存路径。
    """
    # --- 步骤 1: 读取原始文件的所有行 ---
    try:
        print(f"正在读取原始文件: {input_file_path}...")
        with open(input_file_path, 'r', encoding='utf-8') as f:
            lines = f.readlines()
    except FileNotFoundError:
        print(f"错误：找不到输入文件 '{input_file_path}'。请确保文件存在且路径正确。")
        return

    total_lines = len(lines)
    if total_lines == 0:
        print("警告：输入文件为空，无需分割。")
        return
        
    print(f"文件总行数: {total_lines}")

    # --- 步骤 2: 计算分割点 ---
    # 使用 (total_lines + 1) // 2 可以很好地处理奇数和偶数行的情况
    # 例如: 100行 -> 50; 101行 -> 51，第一个文件会多一行
    split_point = (total_lines + 1) // 2
    
    # --- 步骤 3: 分割数据 ---
    part1_lines = lines[:split_point]
    part2_lines = lines[split_point:]

    # --- 步骤 4: 将分割后的数据写入两个新文件 ---
    try:
        # 写入第一个文件
        print(f"正在写入第一个文件 ({len(part1_lines)} 行): {output_file_1_path}")
        with open(output_file_1_path, 'w', encoding='utf-8') as f1:
            f1.writelines(part1_lines)
            
        # 写入第二个文件
        print(f"正在写入第二个文件 ({len(part2_lines)} 行): {output_file_2_path}")
        with open(output_file_2_path, 'w', encoding='utf-8') as f2:
            f2.writelines(part2_lines)

    except Exception as e:
        print(f"写入文件时发生错误: {e}")
        return
        
    print("=" * 30)
    print("文件分割完成！")


# --- 如何使用 ---

if __name__ == "__main__":
    # 1. 请将这里的路径修改为您自己的实际路径
    # 这是上一个脚本生成的jsonl文件
    input_jsonl = "./output_transcription.jsonl" 

    # 2. 定义两个输出文件的名字
    # os.path.splitext可以分割 文件名 和 扩展名
    base_name, extension = os.path.splitext(input_jsonl)
    
    output_part_1 = f"{base_name}_part1{extension}"
    output_part_2 = f"{base_name}_part2{extension}"
    
    # 3. 运行主函数
    split_jsonl_file(input_jsonl, output_part_1, output_part_2)

正在读取原始文件: ./output_transcription.jsonl...
文件总行数: 89560
正在写入第一个文件 (44780 行): ./output_transcription_part1.jsonl
正在写入第二个文件 (44780 行): ./output_transcription_part2.jsonl
文件分割完成！
